## Lab 1. Pandas

### Structured Data Manipulation and Data Wrangling

## Part 1. Data Transformation and Group By Analysis

__Data__: Toronto Parking Tickets dataset `Parking_Tickets_Toronto2020.csv` (original data source https://www.toronto.ca/city-government/data-research-maps/open-data/) describes parking infractions in the City of Toronto issued between April and August 2020.  

Data dictionary:

- date_of_infraction (int) the date of parking violation as YYYYMMDD
- infraction (str) description of the parking violation
- fine (int) parkng ticket amount
- address (str) nearest house number and street to the location where the ticket was issued
- province (str) province or state of origin of the car's license plate


In [1]:
import pandas as pd
import numpy as np
import datetime as dt

#### 1.1- Data Import and Inspection

1. Import CSV data into a pandas data frame
2. Inspect the data frame:
    - how many rows and columns are there?
    - what data types are there?
    - describe the numerical and object columns
    - what is the number of unique values in each column?
3. Print the fist 5 rows of the columns containing character strings ('object' data type)

In [2]:
# 1
df = pd.read_csv("../data/Parking_Tickets_Toronto2020.csv")
df.head()

,date_of_infraction,infraction,fine,address,province
0,20200421,STOP-(ON/OVER) (SIDEWK/FTPATH),150,110 GLENDALE AVE,ON
1,20200421,STOP-(ON/OVER) (SIDEWK/FTPATH),150,108 GLENDALE AVE,ON
2,20200421,PARK ON PRIVATE PROPERTY,30,11 APPLEWOOD LANE,ON
3,20200421,PARK ON PRIVATE PROPERTY,30,11 APPLEWOOD LANE,ON
4,20200421,PARK ON PRIVATE PROPERTY,30,11 APPLEWOOD LANE,ON


In [3]:
#2
# use df.shape, df.dtypes, df.nunique(0), df.describe(), df.describe(include = 'object'))

In [4]:
#3
df.loc[:, df.dtypes == 'object'].head()

,infraction,address,province
0,STOP-(ON/OVER) (SIDEWK/FTPATH),110 GLENDALE AVE,ON
1,STOP-(ON/OVER) (SIDEWK/FTPATH),108 GLENDALE AVE,ON
2,PARK ON PRIVATE PROPERTY,11 APPLEWOOD LANE,ON
3,PARK ON PRIVATE PROPERTY,11 APPLEWOOD LANE,ON
4,PARK ON PRIVATE PROPERTY,11 APPLEWOOD LANE,ON


#### 1.2- Data Wrangling

1. Convert the column "date_of_infraction" into the date-time format
    - use the `apply` method and `strptime` function from the `datetime` package to create a new column `"date"` containing the dates of infractions expressed as a date-time object
    - drop the original `"date_of_infraction"` column from the data frame
    - create two new columns "month" and "week_day" containing the Month and Day of the week extracted from the "date" column. Hint: use the "apply" method and strftime function. Resource:  https://strftime.org/
2. Create a new column `"street"` by extracting street name from the "address" column
    - Hint: the addresses always begin with a house bumber followed by a space folowed by the street name
    - List the top 10 most frequent street names
    - How many unique combinations of address and street are there?
3. Convert the `"week_day"` and `"month"` columns into the Categorical data type
    - also make sure that your categories are properly ordered

In [5]:
# 1
df['date'] = df['date_of_infraction'].astype(str).apply(lambda x: dt.datetime.strptime(x, "%Y%m%d"))
#df = df.drop('date_of_infraction', axis = 1)
df["month"] = df["date"].apply(lambda x: x.strftime("%B"))
df["week_day"] = df["date"].apply(lambda x: x.strftime("%A"))
df.head()

,date_of_infraction,infraction,fine,address,province,date,month,week_day
0,20200421,STOP-(ON/OVER) (SIDEWK/FTPATH),150,110 GLENDALE AVE,ON,2020-04-21,April,Tuesday
1,20200421,STOP-(ON/OVER) (SIDEWK/FTPATH),150,108 GLENDALE AVE,ON,2020-04-21,April,Tuesday
2,20200421,PARK ON PRIVATE PROPERTY,30,11 APPLEWOOD LANE,ON,2020-04-21,April,Tuesday
3,20200421,PARK ON PRIVATE PROPERTY,30,11 APPLEWOOD LANE,ON,2020-04-21,April,Tuesday
4,20200421,PARK ON PRIVATE PROPERTY,30,11 APPLEWOOD LANE,ON,2020-04-21,April,Tuesday


In [6]:
# 2
df['street'] = df['address'].apply(lambda x: x.split(maxsplit = 1)[1])
df.street.value_counts().head(10)

YONGE ST         3797
BAYVIEW AVE      2101
LESLIE ST        1672
COLLEGE ST       1495
CUMBERLAND ST    1481
SPADINA AVE      1426
BAY ST           1278
VICTORIA ST      1260
CHERRY ST        1251
YORKVILLE AVE    1214
Name: street, dtype: int64

In [7]:
df[['address', 'street']].drop_duplicates().shape

(27436, 2)

In [8]:
# 3
df["week_day"] = pd.Categorical(df["week_day"], ordered = True, 
                                categories = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

df['month'] = pd.Categorical(df["month"], ordered = True, 
                             categories = ['April', 'May', 'June', 'July', 'August'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150342 entries, 0 to 150341
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date_of_infraction  150342 non-null  int64         
 1   infraction          150342 non-null  object        
 2   fine                150342 non-null  int64         
 3   address             150342 non-null  object        
 4   province            150342 non-null  object        
 5   date                150342 non-null  datetime64[ns]
 6   month               150342 non-null  category      
 7   week_day            150342 non-null  category      
 8   street              150342 non-null  object        
dtypes: category(2), datetime64[ns](1), int64(2), object(4)
memory usage: 8.3+ MB


#### 1.3- Subsetting and GroupBy Analysis

1. Find the top 5 most frequent infraction categories and top 10 most frequently occurring streets
2. Build a subset of the original data frame where the infractions and streets are those you identified in step 1
    - How many rows does the subset data frame contain?
2. Using the subset data from step 2 and the groupby method, compute:
    - mean fines for each month
    - mean fines for each week day
    - find the provinces which paid a total of > 10000 in parking tickets

In [9]:
# 1
df['infraction'].value_counts().index[:5]

Index(['PARK ON PRIVATE PROPERTY', 'PARK-SIGNED HWY-PROHIBIT DY/TM',
       'PARK MACHINE-REQD FEE NOT PAID', 'STOP-SIGNED HWY-PROHIBIT TM/DY',
       'PARK ON MUNICIPAL PROPERTY'],
      dtype='object')

In [10]:
df['street'].value_counts().index[:10]

Index(['YONGE ST', 'BAYVIEW AVE', 'LESLIE ST', 'COLLEGE ST', 'CUMBERLAND ST',
       'SPADINA AVE', 'BAY ST', 'VICTORIA ST', 'CHERRY ST', 'YORKVILLE AVE'],
      dtype='object')

In [11]:
# 2
df2 = df[(df['street'].isin(list(df['street'].value_counts().index[:10]))) &
        (df['infraction'].isin(list(df['infraction'].value_counts().index[:5])))]

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12056 entries, 5 to 150341
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date_of_infraction  12056 non-null  int64         
 1   infraction          12056 non-null  object        
 2   fine                12056 non-null  int64         
 3   address             12056 non-null  object        
 4   province            12056 non-null  object        
 5   date                12056 non-null  datetime64[ns]
 6   month               12056 non-null  category      
 7   week_day            12056 non-null  category      
 8   street              12056 non-null  object        
dtypes: category(2), datetime64[ns](1), int64(2), object(4)
memory usage: 777.6+ KB


In [12]:
# 3
df2.groupby("month").mean('fine')
## also try df2.groupby("month")['fine'].mean()

,date_of_infraction,fine
month,,
April,2.020043e+07,36.914062
May,2.020052e+07,34.662096
June,2.020062e+07,34.228284
July,2.020072e+07,39.636121
August,2.020081e+07,40.491900


In [13]:
df2.groupby("week_day").mean('fine')

,date_of_infraction,fine
week_day,,
Monday,2.020071e+07,38.500252
Tuesday,2.020071e+07,40.295675
Wednesday,2.020071e+07,39.382889
Thursday,2.020072e+07,40.186047
Friday,2.020071e+07,37.365510
Saturday,2.020069e+07,35.969388
Sunday,2.020068e+07,35.735493


In [14]:
fines_by_province = df.groupby("province").sum('fine')
fines_by_province[fines_by_province['fine'] > 10000]

,date_of_infraction,fine
province,,
AB,9554933211,21390
AZ,4363353931,14230
MB,4040141357,10440
NS,4363348191,11340
ON,2934334378088,7454860
PQ,6019798099,12290
QC,50724020982,149820


## Part 2.  Merging. Missing data

#### Data:

- `debt_public.csv`
    - this data table contains the following columns:
        - Country
        - gross_debt_per_GDP (gross government debt as percent of GDP)
        - net_debt_per_GDP (net government debt as percent of GDP)


- `gdp_by_country.csv`
    - gross domestic product estimates from three independent sources (IMF, WB, CIA) and the year of the estimate


- `continents.csv`
    - a table of countries and continents


#### 2.1- Import and Inspect Data

1. Import data from the following sources: `debt_public.csv`, `gdp_by_country.csv`, `continents.csv` into Pandas data frames
2. Inspect the data frames:
    - Preview a few sample rows
    - Preview and inspect descriptive statistics for the numerical and string columns
3. Does any of the three data frames contain missing data (`NaN`)?

In [15]:
# 1

pDebt = pd.read_csv("../data/debt_public.csv")
gdp = pd.read_csv("../data/gdp_by_country.csv")
continents = pd.read_csv("../data/continents.csv")

In [16]:
# 2

## Hint: head(), describe()

In [17]:
# 3
print(
    pDebt.isna().sum(), '\n', '\n',
    gdp.isna().sum(), '\n','\n',
    continents.isna().sum(),'\n', '\n')

Country                 0
gross_debt_per_GDP      8
net_debt_per_GDP      110
dtype: int64 
 
 Country          0
IMF_Estimate    36
IMF_Year        36
WB_Estimate     30
WB_Year         30
CIA_Estimate     0
CIA_Year         0
dtype: int64 
 
 Country      0
Continent    0
dtype: int64 
 



#### 2.2- Data Transformation

1. gdp data:
    - remove all except 'Country' and 'CIA_Estimate'
    - rename 'CIA_Estimate' column to 'GDP'
    - add a `"Continent"` column by merging the `gdp` and `continents` data frames on the `"Country"` column


2. public debt data:
    - remove rows where `"gross_debt_per_GDP"` is missing (`NaN`)
    - check for the number of missing values in each column
    - merge the `pDebt` and `gdp` data frames into a master data frame `df_debt`
    - add a caclulated column `"gross_pub_debt"` for the absolute values of gross public debt using the GDP values and the values of gross debt expressed as percentage of GDP
    - add another calculated column `"debt_bin"` by binning "gross_pub_debt" into two bins: "Low" and "High" separated by the median value of `"gross_pub_debt"`

In [18]:
# # 1
gdp = gdp[["Country", "CIA_Estimate"]]
gdp.columns = ["Country", "GDP"]
gdp = pd.merge(gdp, continents, on = "Country")

In [19]:
# #2

pDebt = pDebt.dropna(subset = ['gross_debt_per_GDP'])
pDebt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190 entries, 1 to 197
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             190 non-null    object 
 1   gross_debt_per_GDP  190 non-null    float64
 2   net_debt_per_GDP    88 non-null     float64
dtypes: float64(2), object(1)
memory usage: 5.9+ KB


In [20]:
df_debt = pd.merge(gdp, pDebt)

In [21]:
df_debt['gross_pub_debt'] = (0.01 *
        df_debt['gross_debt_per_GDP'] * df_debt['GDP']).astype(int)

In [22]:
df_debt['debt_bin'] = pd.cut(df_debt["gross_pub_debt"], bins =
                              [0, df_debt['gross_pub_debt'].median(), df_debt['gross_pub_debt'].max()],
                             labels = ["Low", "High"])

#### 2.3- impute missing values
- verify again which columns have missing values in the master data frame `df_debt`
- does the number of missing values justify using .dropna() (removing entire rows containing missing values)?
- check which fillna method should be used. One possibility is to fill the NaNs with the mean of the non-missing values - this can work if the distribution is reasonably symmetrical, i.e., mean and median are close to each other

In [23]:
# #3
df_debt.info()
### too many rows with missing values
### .dropna() will cause significant data loss

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 175
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Country             176 non-null    object  
 1   GDP                 176 non-null    int64   
 2   Continent           176 non-null    object  
 3   gross_debt_per_GDP  176 non-null    float64 
 4   net_debt_per_GDP    84 non-null     float64 
 5   gross_pub_debt      176 non-null    int32   
 6   debt_bin            176 non-null    category
dtypes: category(1), float64(2), int32(1), int64(1), object(2)
memory usage: 9.2+ KB


In [24]:
df_debt['net_debt_per_GDP'].describe()
## yes we can fillna with mean values

count     84.000000
mean      49.747619
std       35.705615
min      -86.700000
25%       31.125000
50%       47.800000
75%       64.725000
max      168.100000
Name: net_debt_per_GDP, dtype: float64

In [25]:
df_debt['net_debt_per_GDP'] = df_debt['net_debt_per_GDP'].fillna(df_debt['net_debt_per_GDP'].mean())
df_debt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 175
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Country             176 non-null    object  
 1   GDP                 176 non-null    int64   
 2   Continent           176 non-null    object  
 3   gross_debt_per_GDP  176 non-null    float64 
 4   net_debt_per_GDP    176 non-null    float64 
 5   gross_pub_debt      176 non-null    int32   
 6   debt_bin            176 non-null    category
dtypes: category(1), float64(2), int32(1), int64(1), object(2)
memory usage: 9.2+ KB


In [26]:
df_debt.head()

,Country,GDP,Continent,gross_debt_per_GDP,net_debt_per_GDP,gross_pub_debt,debt_bin
0,China,23009780,Asia,71.5,49.747619,16451992,High
1,United States,19846720,Americas,128.1,99.600000,25423648,High
2,India,8443360,Asia,84.2,49.747619,7109309,High
3,Japan,5224850,Asia,262.5,168.100000,13715231,High
4,Germany,4238800,Europe,69.6,47.000000,2950204,High
